In [24]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
minst = input_data.read_data_sets('MINST_data', one_hot = True)

Extracting MINST_data/train-images-idx3-ubyte.gz
Extracting MINST_data/train-labels-idx1-ubyte.gz
Extracting MINST_data/t10k-images-idx3-ubyte.gz
Extracting MINST_data/t10k-labels-idx1-ubyte.gz


In [3]:
def Weight(shape):
    weight_mat = tf.truncated_normal(shape, stddev = 0.1)
    return tf.Variable(weight_mat)
def Bias(shape):
    Bias_mat = tf.constant(0.1, shape = shape)
    return tf.Variable(Bias_mat)

&emsp;将图像转换为四维张量[-1, 28, 28, 1],-1表示样本数量不定,28x28是图片尺寸,最后是通道数

In [5]:
xs = tf.placeholder(tf.float32, [None, 784])
ys = tf.placeholder(tf.float32, [None, 10])
x_image = tf.reshape(xs, [-1, 28, 28, 1])

&emsp;第一层卷积层与第一层池化层  
&emsp;5x5为卷积核大小,1为通道数,32为卷积核数目即卷积特征数也即输出通道数  
&emsp;strides为卷积步长,第一个与第四个元素必须为1，中间两个元素为横纵向的步长  
&emsp;ksize为池化块大小,padding为填充模式,“VALID”就是不填充,“SAME”就是周围填0,填充使得图片边缘的像素也能卷积  
&emsp;max_pool是最大值池化,区域内取最大值

In [10]:
Weight1 = Weight([5, 5, 1, 32])
Bias1 = Bias([32])
con1 = tf.nn.relu(tf.nn.conv2d(x_image, Weight1, strides = [1, 1, 1, 1], padding = "SAME") + Bias1)
pool1 = tf.nn.max_pool(con1, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = "SAME")

&emsp;第二层卷积层与第二层池化层  
&emsp;多通道的卷积是在同一个位置,对每个通道的图像卷积,最后相加输出一个通道的特征

In [11]:
Weight2 = Weight([5, 5, 32, 64])
Bias2 = Bias([64])
con2 = tf.nn.relu(tf.nn.conv2d(pool1, Weight2, strides = [1, 1, 1, 1], padding = "SAME") + Bias2)
pool2 = tf.nn.max_pool(con2, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = "SAME")

&emsp;全连接层,输入是卷积池化后的图像大小,第一次28x28x1->14x14x32,第二次14x14x32->7x7x64

In [14]:
fc_Weight = Weight([7 * 7 * 64, 1024])
fc_Bias = Bias([1024])
fc_in = tf.reshape(pool2, [-1, 7 * 7 * 64])
fc = tf.nn.relu(tf.matmul(fc_in, fc_Weight) + fc_Bias)

&emsp;Drop_out防止过拟合

In [15]:
keep_prob = tf.placeholder("float")
fc_drop = tf.nn.dropout(fc, keep_prob)

&emsp;加入softmax层

In [21]:
softmax_w = Weight([1024,10])
softmax_bias = Bias([10])
y = tf.nn.softmax(tf.matmul(fc_drop, softmax_w) + softmax_bias)

&emsp;定义交叉熵损失函数

In [22]:
cross_entropy = -tf.reduce_mean(ys * tf.log(y))
train = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

In [25]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(ys,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

In [56]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for step in range(10000):
        batch = mnist.train.next_batch(50)
        sess.run(train, feed_dict = {xs:batch[0], ys:batch[1], keep_prob:1.0})
        if step % 100 == 0:
            print(sess.run(accuracy, feed_dict = {xs:batch[0], ys:batch[1], keep_prob:1.0}))
    print("test: ", sess.run(accuracy, feed_dict = {xs:minst.test.images, ys:minst.test.labels, keep_prob:1.0}))

0.16
0.9
0.8
0.98
0.88
0.98
0.86
0.94
0.9
0.94
0.96
0.98
0.98
0.98
1.0
0.96
0.96
0.94
0.98
1.0
1.0
1.0
0.96
0.98
0.96
1.0
1.0
1.0
1.0
0.96
0.98
1.0
0.98
0.96
0.98
0.98
0.96
1.0
1.0
0.98
0.98
1.0
1.0
1.0
1.0
1.0
0.98
1.0
0.98
1.0
0.98
1.0
1.0
0.98
0.98
1.0
1.0
1.0
1.0
0.98
1.0
1.0
0.98
1.0
0.98
1.0
1.0
0.98
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.98
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
test:  0.991
